In [2]:
from __future__ import print_function
import sys
sys.path.append('../')
sys.path.append('/')
import argparse
import os
import random
import torch
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from tqdm import tqdm
import numpy as np
import pdb
from glob import glob
import pandas as pd
from metrics import AverageMeter
import time
import wandb
from pytorch_lightning.core.lightning import LightningModule
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import Callback
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks import GradientAccumulationScheduler

ModuleNotFoundError: No module named 'metrics'

In [4]:
import wandb

In [28]:
wandb.init(project='point_ring_light_vs_pointnet5c_simpledataset',name='test_run')

W&B Run: https://app.wandb.ai/gufengy/point_ring_light_vs_pointnet5c_simpledataset/runs/2pv977ib

In [27]:
pretrained_model_path = os.path.join("ckpt/point_ring_light_vs_pointnet5c_simpledataset",'best_model.pth')
package = torch.load(pretrained_model_path)

FileNotFoundError: [Errno 2] No such file or directory: 'ckpt/point_ring_light_vs_pointnet5c_simpledataset/best_model.pth'

In [13]:
package.keys()

dict_keys(['state_dict', 'scheduler', 'optimizer', 'epoch', 'conda_env', 'notification_email', 'dataset_root', 'num_workers', 'phase', 'num_points', 'load_pretrain', 'synchonization', 'tol_stop', 'num_gpu', 'num_channel', 'batch_size', 'epoch_max', 'wd_project', 'debug', 'model', 'including_ring', 'apex', 'opt_level', 'gpu_list', 'time', 'epoch_ckpt', 'val_miou', 'model_name', 'save_root', 'Miou_validation_ave', 'Biou_validation_ave', 'Fiou_validation_ave', 'loss_validation_ave', 'OA_validation_ave', 'time_complexicity_validation_ave', 'storage_complexicity_validation_ave', 'name'])

In [40]:
import wandb
wandb.init(project="preemptable", resume=True)
package = wandb.restore('best_model.pth',run_path = 'gufengy/point_ring_light_vs_pointnet5c_simpledataset/tn6zxc7c')

Streaming file created twice in same run: /media/malcolm/1E577EB53AA8D6D4/Big_red_framework/wandb/run-20200523_124434-2eu2aqbn/wandb-events.jsonl


In [55]:
torch.load('ckpt/Pointnet_ring_light4c_upsample+groupConv/saves/val_miou0.8188_Epoch119.pth')['epoch']

119

In [53]:
torch.load(package.name)['epoch']

99

In [16]:
package = torch.load(pretrained_model_path)

In [18]:
os.path.exists(pretrained_model_path)

True

In [25]:
def load_pretrained(opt):
    print('----------------------loading Pretrained----------------------')
    pretrained_model_path = os.path.join(opt.load_pretrain,'best_model.pth')
    package = torch.load(pretrained_model_path)
    para_state_dict = package['state_dict']
    opt.num_channel = package['num_channel']
    opt.time = package['time'] 
    opt.epoch_ckpt = package['epoch']
    opt.val_miou = package['Validation_ave_miou'] 
    state_dict = convert_state_dict(para_state_dict)
    ckpt_,ckpt_file_name  = opt.load_pretrain.split("/")
    module_name = ckpt_+'.'+ckpt_file_name+'.'+'model'
    MODEL = importlib.import_module(module_name)
    model = MODEL.get_model(input_channel = opt.num_channel)
    Model_Specification = MODEL.get_model_name(input_channel = opt.num_channel)
    f_loss = MODEL.get_loss(input_channel = opt.num_channel)
    opt.model = Model_Specification[:-3]


    print('----------------------Model Info----------------------')
    print('Root of prestrain model: ', pretrained_model_path)
    print('Model: ', opt.model)
    print('Model Specification: ', Model_Specification)
    print('Trained Date: ',opt.time)
    print('num_channel: ',opt.num_channel)
    name = input("Edit the name or press ENTER to skip: ")
    if(name!=''):
        package['name'] = name
    print('Pretrained model name: ', package['name'])
    save_model(package,pretrained_model_path)     
    model.load_state_dict(state_dict)
    model = torch.nn.DataParallel(model)
    model.cuda()
    f_loss.cuda()

    print('----------------------Configure optimizer and scheduler----------------------')
    optimizer = (package['optimizer'])
    scheduler = (package['scheduler'])

    return opt,model,f_loss,optimizer,scheduler


'/media/malcolm/1E577EB53AA8D6D4/Big_red_framework/wandb/run-20200523_114841-3hrjo2og'

In [29]:
wandb.restore('best_model.pth',run_path = 'gufengy/point_ring_light_vs_pointnet5c_simpledataset/3j1kmdwt')

AttributeError: File best_model.pth does not exist.

In [ ]:
from pytorch_lightning.callbacks import LearningRateLogger
class LitProgressBar(ProgressBar):
def init_validation_tqdm(self):
bar = super().init_validation_tqdm()
bar.set_description('running validation ...')
return bar
bar = LitProgressBar()
trainer = Trainer(callbacks=[bar])

In [14]:

class MyPrintingCallback(Callback):
    def on_init_start(self, trainer):
        print('Starting to init trainer!')
    def on_init_end(self, trainer):
        print('trainer is init now')
    def on_train_end(self, trainer, pl_module):
        print('do something when training ends')


In [15]:
on_batch_end(trainer, pl_module)
Called when the training batch ends.
on_batch_start(trainer, pl_module)
Called when the training batch begins.
on_epoch_end(trainer, pl_module)
Called when the epoch ends.
on_epoch_start(trainer, pl_module)
Called when the epoch begins.
on_init_end(trainer)
Called when the trainer initialization ends, model has not yet been set.
on_init_start(trainer)
Called when the trainer initialization begins, model has not yet been set.
on_sanity_check_end(trainer, pl_module)
Called when the validation sanity check ends.
on_sanity_check_start(trainer, pl_module)
Called when the validation sanity check starts.
on_test_batch_end(trainer, pl_module)
Called when the test batch ends.
on_test_batch_start(trainer, pl_module)
Called when the test batch begins.
on_test_end(trainer, pl_module)
Called when the test ends.
on_test_start(trainer, pl_module)
Called when the test begins.
on_train_end(trainer, pl_module)
Called when the train ends.
on_train_start(trainer, pl_module)
Called when the train begins.
on_validation_batch_end(trainer, pl_module)
Called when the validation batch ends.
on_validation_batch_start(trainer, pl_module)
Called when the validation batch begins.
on_validation_end(trainer, pl_module)
Called when the validation loop ends.
on_validation_start(trainer, pl_module)
Called when the validation loop begins.
trainer = Trainer(callbacks=[MyPrintingCallback()])

INFO:lightning:GPU available: True, used: False


Starting to init trainer!
trainer is init now


In [9]:
class LitModel(LightningModule):
    def __init__(self):
        super().__init__()
        self.l1 = torch.nn.Linear(28 * 28, 10)
    def forward(self, x):
        return torch.relu(self.l1(x.view(x.size(0), -1)))
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        tensorboard_logs = {'train_loss': loss}
        return {'loss': loss, 'log': tensorboard_logs}
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)
    def train_dataloader(self):
        dataset = MNIST(os.getcwd(), train=True, download=True, transform=transforms.
        ˓→ToTensor())
        loader = DataLoader(dataset, batch_size=32, num_workers=4, shuffle=True)
        return loader
    @staticmethod
    def add_model_specific_args(parent_parser):
        parser = ArgumentParser(parents=[parent_parser], add_help=False)
        parser.add_argument('--encoder_layers', type=int, default=12)
        parser.add_argument('--data_path', type=str, default='/some/path')
        return parser

In [ ]:
from argparse import ArgumentParser
parser = ArgumentParser()
# add PROGRAM level args
parser.add_argument('--conda_env', type=str, default='some_name')
parser.add_argument('--notification_email', type=str, default='will@email.com')
# add model specific args
parser = LitModel.add_model_specific_args(parser)

In [ ]:
model = LitModel()
# most basic trainer, uses good defaults
trainer = Trainer(gpus=8, num_nodes=1)
trainer.fit(model)

In [ ]:
class LitModel(LightningModule):
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        return {'val_loss': F.cross_entropy(y_hat, y)}
    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        tensorboard_logs = {'val_loss': avg_loss}
        return {'val_loss': avg_loss, 'log': tensorboard_logs}
    def val_dataloader(self):
        # TODO: do a real train/val split
        dataset = MNIST(os.getcwd(), train=False, download=True, transform=transforms.ToTensor())
        loader = DataLoader(dataset, batch_size=32, num_workers=4)
    return loader

In [ ]:
class LitModel(LightningModule):
    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        return {'test_loss': F.cross_entropy(y_hat, y)}
    def test_epoch_end(self, outputs):
        avg_loss = torch.stack([x['test_loss'] for x in outputs]).mean()
        tensorboard_logs = {'test_loss': avg_loss}
        return {'avg_test_loss': avg_loss, 'log': tensorboard_logs}
    def test_dataloader(self):
        # TODO: do a real train/val split
        dataset = MNIST(os.getcwd(), train=False, download=True, transform=transforms.ToTensor())
        loader = DataLoader(dataset, batch_size=32, num_workers=4)
    return loader